<a href="https://colab.research.google.com/github/SalmaSherif7070/Sensors-Monitoring-Data-Governance-Project/blob/main/Sensors_Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:
!pip install pandera[io]
!pip install Cerberus
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 21.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for petl: filename=petl-1.7.15-py3-none-any.whl size=230718 sha256=f84da087710005212bf36f705af46cf68d9b3c9cb41aef0d8233d314b1ddb597
  Stored in directory: /root/.cache/pip/wheels/25/98/91/62fbf9f9cb32952d7d91532152b893f7eab007f6aea750a3f5
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=35

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import json
import pandera as pa
from pandera import Column, DataFrameSchema, Check, Index
from cerberus import Validator
from cerberus import Validator
from cryptography.fernet import Fernet
import matplotlib.pyplot as plt
import seaborn as sns

# Read & prepare Dataframs

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CSV file/predictive-maintenance-dataset-corrupted-data.csv')
df

,ID,revolutions,humidity,vibration,x1,x2,x3,x4,x5
0,1.0,93.744,73.999,18.0,167.743,19.745,1.266828,NaN,5475.852001
1,2.0,93.740,73.999,18.0,167.739,19.741,1.266774,8787.187600,5475.852001
2,3.0,93.736,73.998,18.0,167.734,19.738,1.266737,8786.437696,5475.704004
3,4.0,93.732,73.998,18.0,167.730,19.734,1.266683,8785.687824,5475.704004
4,5.0,93.729,73.998,18.0,167.727,19.731,1.266642,8785.125441,5475.704004
...,...,...,...,...,...,...,...,...,...
112496,496.0,91.834,73.838,18.5,165.672,17.996,1.243723,8433.483556,5452.050244
112497,497.0,91.830,73.837,18.5,165.667,17.993,1.243685,NaN,5451.902569
112498,498.0,91.827,73.837,18.5,165.664,17.990,1.243645,8432.197929,5451.902569
112499,499.0,91.823,73.837,18.5,165.660,17.986,1.243591,-8431.463329,5451.902569


# Finding the correlation between the several columns

Finding the correlation between the several columns to know the dependences and to enhace the dataset understanding

In [ ]:
correlation_matrix = df[['x1', 'x2', 'x3', 'x4', 'revolutions',	'humidity',	'vibration']].corr()
print("Correlation Matrix:\n", correlation_matrix)

Correlation Matrix:
                    x1        x2        x3        x4  revolutions  humidity  \
x1           1.000000  0.997509  0.998019  0.572729     0.999373 -0.069510   
x2           0.997509  1.000000  0.999936  0.576670     0.999382 -0.139740   
x3           0.998019  0.999936  1.000000  0.576240     0.999603 -0.131726   
x4           0.572729  0.576670  0.576240  1.000000     0.574977 -0.118767   
revolutions  0.999373  0.999382  0.999603  0.574977     1.000000 -0.104891   
humidity    -0.069510 -0.139740 -0.131726 -0.118767    -0.104891  1.000000   
vibration   -0.113228 -0.122018 -0.120294 -0.085224    -0.117725  0.137225   

             vibration  
x1           -0.113228  
x2           -0.122018  
x3           -0.120294  
x4           -0.085224  
revolutions  -0.117725  
humidity      0.137225  
vibration     1.000000  


# Cleaning

In [ ]:
# Duplicates handeling
duplicate = df[df.duplicated()]
print("Before drop_duplicates \n", duplicate.sum())
df = df.drop_duplicates()
print("\n After drop_duplicates \n", df.duplicated())

Before drop_duplicates 
 ID             7.006300e+04
revolutions    2.626034e+04
humidity       2.091897e+04
vibration      5.162960e+03
x1             4.717931e+04
x2             5.341365e+03
x3             3.552588e+02
x4             2.179487e+06
x5             1.546302e+06
dtype: float64

 After drop_duplicates 
 0         False
1         False
2         False
3         False
4         False
          ...  
112493    False
112494    False
112497    False
112499    False
112500    False
Length: 112218, dtype: bool


In [ ]:
print(df.isnull().sum())

for col in df.columns:
  df[col] = df[col].fillna(df[col].mean())

print('\n', df.isnull().sum())

ID                 7
revolutions        4
humidity           8
vibration       2443
x1                 7
x2                10
x3                 3
x4             23692
x5                 8
dtype: int64

 ID             0
revolutions    0
humidity       0
vibration      0
x1             0
x2             0
x3             0
x4             0
x5             0
dtype: int64


<ipython-input-7-bb1f61383086>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mean())


In [ ]:
# Convert the corrupted cells back to numeric values
for col in df.columns:
    df.loc[:, col] = pd.to_numeric(df.loc[:, col], errors='coerce')

In [ ]:
# Convert all negative values in x4 column to positive
df['x4'] = df['x4'].abs()

# Convert all IDs to int
df["ID"] = df["ID"].astype(int)

<ipython-input-9-f95737709102>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x4'] = df['x4'].abs()
<ipython-input-9-f95737709102>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ID"] = df["ID"].astype(int)


In [ ]:
numerical_columns = ["revolutions", "humidity", "vibration", "x1", "x2", "x3", "x4", "x5"]

Q1 = df[numerical_columns].quantile(0.25)
Q3 = df[numerical_columns].quantile(0.75)
IQR = Q3-Q1
print('IQR Scores of columns:\n',IQR)

IQR Scores of columns:
 revolutions      34.262500
humidity          0.816000
vibration        30.780000
x1               34.199500
x2               34.454750
x3                0.464731
x4             2613.445387
x5              121.293504
dtype: float64


# Validation

In [ ]:
schema = pa.DataFrameSchema(
    columns={
        "ID": pa.Column(pa.Int, nullable=False),
        "revolutions": pa.Column(pa.Float, nullable=False),
        "humidity": pa.Column(pa.Float, nullable=False),
        "vibration": pa.Column(pa.Float, nullable=False),
        "x1": pa.Column(pa.Float, nullable=False),
        "x2": pa.Column(pa.Float, nullable=False),
        "x3": pa.Column(pa.Float, nullable=False),
        "x4": pa.Column(pa.Float, nullable=True),
        "x5": pa.Column(pa.Float, nullable=False),
    }
)

validated_df = schema(df)
validated_df

,ID,revolutions,humidity,vibration,x1,x2,x3,x4,x5
0,1,93.744,73.999,18.00,167.743,19.745,1.266828,1933.402999,5475.852001
1,2,93.740,73.999,18.00,167.739,19.741,1.266774,8787.187600,5475.852001
2,3,93.736,73.998,18.00,167.734,19.738,1.266737,8786.437696,5475.704004
3,4,93.732,73.998,18.00,167.730,19.734,1.266683,8785.687824,5475.704004
4,5,93.729,73.998,18.00,167.727,19.731,1.266642,8785.125441,5475.704004
...,...,...,...,...,...,...,...,...,...
112493,493,91.846,73.839,18.49,165.685,18.007,1.243868,8435.687716,5452.197921
112494,494,91.842,73.838,18.49,165.680,18.004,1.243831,1933.402999,5452.050244
112497,497,91.830,73.837,18.50,165.667,17.993,1.243685,1933.402999,5451.902569
112499,499,91.823,73.837,18.50,165.660,17.986,1.243591,8431.463329,5451.902569


In [ ]:
try:
    schema.validate(df, lazy=True)
except pa.errors.SchemaErrors as exc:
    failure_cases_df = exc.failure_cases
    display(exc.failure_cases)

# Privacy and Security Policies

In [ ]:
class MACPolicy:
    def __init__(self):
        self.access_rules = {}

    def add_access_rule(self, role, resource, action):
        if resource not in self.access_rules:
            self.access_rules[resource] = {}
        if action not in self.access_rules[resource]:
            self.access_rules[resource][action] = []
        self.access_rules[resource][action].append(role)

    def check_access(self, subject, resource, action):
        if resource in self.access_rules and action in self.access_rules[resource]:
            roles_with_access = self.access_rules[resource][action]
            return subject in roles_with_access
        return False

In [ ]:
mac_policy = MACPolicy()

mac_policy.add_access_rule("staf", "/content/drive/MyDrive/Colab Notebooks/CSV file/predictive-maintenance-dataset-corrupted-data.csv", "read")
mac_policy.add_access_rule("farmer", "/content/drive/MyDrive/Colab Notebooks/CSV file/predictive-maintenance-dataset-corrupted-data.csv", "write")

print("Access for staf to read sensor data:", mac_policy.check_access("staf", "/content/drive/MyDrive/Colab Notebooks/CSV file/predictive-maintenance-dataset-corrupted-data.csv", "read"))
print("Access for farmer to write sensor data:", mac_policy.check_access("farmer", "/content/drive/MyDrive/Colab Notebooks/CSV file/predictive-maintenance-dataset-corrupted-data.csv", "write"))

Access for staf to read sensor data: True
Access for farmer to write sensor data: True


In [ ]:
df

,ID,revolutions,humidity,vibration,x1,x2,x3,x4,x5
0,1,93.744,73.999,18.00,167.743,19.745,1.266828,1933.402999,5475.852001
1,2,93.740,73.999,18.00,167.739,19.741,1.266774,8787.187600,5475.852001
2,3,93.736,73.998,18.00,167.734,19.738,1.266737,8786.437696,5475.704004
3,4,93.732,73.998,18.00,167.730,19.734,1.266683,8785.687824,5475.704004
4,5,93.729,73.998,18.00,167.727,19.731,1.266642,8785.125441,5475.704004
...,...,...,...,...,...,...,...,...,...
112493,493,91.846,73.839,18.49,165.685,18.007,1.243868,8435.687716,5452.197921
112494,494,91.842,73.838,18.49,165.680,18.004,1.243831,1933.402999,5452.050244
112497,497,91.830,73.837,18.50,165.667,17.993,1.243685,1933.402999,5451.902569
112499,499,91.823,73.837,18.50,165.660,17.986,1.243591,8431.463329,5451.902569


In [ ]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
cipher = Fernet(key)
df['x1'] = df['x1'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
df['x2'] = df['x2'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
df['x3'] = df['x3'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
df['x4'] = df['x4'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
df

<ipython-input-16-ef0177bbc960>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x1'] = df['x1'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
<ipython-input-16-ef0177bbc960>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x2'] = df['x2'].astype(str).apply(lambda x: cipher.encrypt(x.encode()).decode())
<ipython-input-16-ef0177bbc960>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,ID,revolutions,humidity,vibration,x1,x2,x3,x4,x5
0,1,93.744,73.999,18.00,gAAAAABmRdKMjwrVglgLBwkF9BGrYw0yMVtqUB5yyqZ8lv...,gAAAAABmRdKaoEbrQw8EIrY-3GRST7XEkUP53TIDDMNkvX...,gAAAAABmRdKjvHTxuPiVGFbD_HNCTxkayGqoPC_JAh5p9g...,gAAAAABmRdKqxvQknNS8RXKa5GSf3WqgyBUSkF6sa0awAd...,5475.852001
1,2,93.740,73.999,18.00,gAAAAABmRdKMF0YQyHmJ7kntkk1rABgZx7pGzMoEd2jWyx...,gAAAAABmRdKad_s3jyU0cWKH2tqSfrL4Qm2mRlw-UW0D50...,gAAAAABmRdKjOgI1W9h229_Ca_Licmusmj-u7On8I2uI2i...,gAAAAABmRdKqXpdyvOAccRcsZcxK6lgatEgaWXLASUmEoB...,5475.852001
2,3,93.736,73.998,18.00,gAAAAABmRdKMV_N78ocRIOM1Lo2xjZLiZdcEOlCj9cmQQH...,gAAAAABmRdKaYfFPsxCBoWVOkS2aAOusH1B3rbMhd5KHAj...,gAAAAABmRdKj5PD8jHTcespzYNlsvrcqxjFnoBQLbbH_5c...,gAAAAABmRdKqLcY9T3M46k3M09GsPZcxLWxCBiuAkbH7iJ...,5475.704004
3,4,93.732,73.998,18.00,gAAAAABmRdKM7PLqsOhnGg71K1kxoycK521sC-lrOSPUek...,gAAAAABmRdKazbnD5HRnKWBTVNXr5o87SYo-7ip7n4xIqw...,gAAAAABmRdKjA49158jfhKJlpWu6Xk2_NfN_sd8PlJsuuI...,gAAAAABmRdKq2-in17I1FTAvklGichLWMsynd30_rM28bs...,5475.704004
4,5,93.729,73.998,18.00,gAAAAABmRdKM-G7c5RfoGQ1HFpITzQ0pHg0UZKoN2Tolvt...,gAAAAABmRdKak5D2T2C8b82gm3jg-aAQIX7xzdIjlIaj6o...,gAAAAABmRdKjY_ivKflTq3jBrIiUqmu76RNFGvdQyzv64A...,gAAAAABmRdKqNLKPn0W4VZMCbvB5WKWeItoKRpashF_rCa...,5475.704004
...,...,...,...,...,...,...,...,...,...
112493,493,91.846,73.839,18.49,gAAAAABmRdKalFa5is8Kn3UexdA3HAi48q0yhRvwNrhqpy...,gAAAAABmRdKjVRZP81sLSsKB0_RU-Z0Ayr60OkyVMzv-XX...,gAAAAABmRdKqMpEfXW1JDVYnD0XuqZ3dp9nJk8ISxdnlyJ...,gAAAAABmRdK0cgf1m5aUHMbqJkiXRRqdAsbyoU4QkkHX6r...,5452.197921
112494,494,91.842,73.838,18.49,gAAAAABmRdKaee9DaVecd4ve9oqVQgOBFjKWwXiQlfvV9c...,gAAAAABmRdKjw_-HP_29U-UntFye_hQ6ffdJLSnHh8aQYT...,gAAAAABmRdKqYrsBfZ2sdZTFfQiaDWt7GK6nh6JaBB6FUI...,gAAAAABmRdK0YU-orjw0TUG_FJrgAE63t96OU8kWpxbCcJ...,5452.050244
112497,497,91.830,73.837,18.50,gAAAAABmRdKaAzLJMgpYb5OU4xdD7NNDlyXsSzKSE-L0NJ...,gAAAAABmRdKjDBBqQu2U5uFiharhx4vwCGocCBxs24TUcI...,gAAAAABmRdKqUyNS36MnLtJKnROFgaSWFWJ-10tFBd9a6g...,gAAAAABmRdK0v35MWsRGIorr4ZTTje-j7pTbmfRXmtMeJ4...,5451.902569
112499,499,91.823,73.837,18.50,gAAAAABmRdKaCdoSmaOyQwQzuP56y26IQQQr2ZTmG1fGmO...,gAAAAABmRdKjOCA_dchdp9O9bbNU7YE8WolnrpwTEqMv7Y...,gAAAAABmRdKqF-yl__YYfD7wAlkQyZjtjUheazlCGikO7_...,gAAAAABmRdK0gDYx8MKFrer1XvbwM0h3XBrjkSlYk3LoKW...,5451.902569
